# Covid-19 Data Analysis using SQL

## Introduction

__Problem Context__

The goal of this project is to analyze data about the COVID-19 pandemic on the continent of Africa. The analysis will try generate general statistics about how the pandemic affected different countries. Some of the questions that will be answered are:
1. Countries with the highest deaths count
2. Countries where one was most likely to die if contaminated
3. Countries leading the vaccination efforts

and more as the project continues.

__Analytical Context__

This analysis will be done using a SQLite database. Different levels of understanding of SQL will be displayed, but the main ones will be:
- Querying Table
- Aggregating Data
- Joining Related Tables
- Computing Ratios

<h2>Overview of the Data</h2>

The data for this project is contained in the `covid_project.db` SQLite database. The database has two tables __covid_deaths__ and __covid_vaccinations__. The data was downloaded has a `csv` file from [Our World in Data](https://ourworldindata.org/covid-deaths) then loaded into __DB Browser for SQLite__ to produce the database used here. The data was acquired on September 14, 2021.

__Import the Libraries and the `sql` extension__

In [1]:
import pandas as pd
from sqlalchemy import create_engine
import warnings
warnings.filterwarnings('ignore')

__Load in the database__

In [2]:
# create a database engine
engine = create_engine("sqlite:///data/covid_project.db")

In [3]:
# see table names
table_names = engine.table_names()
table_names

['covid_deaths', 'covid_vaccinations']

In [4]:
# print first 5 rows of covid_deaths table 
query = """
    SELECT *
    FROM covid_deaths
    WHERE continent = 'Africa'
    LIMIT 5;
"""

with engine.connect() as con:
    rs = con.execute(query)
    df = pd.DataFrame(rs.fetchall())
    df.columns = rs.keys()
    
df

,iso_code,continent,location,date,population,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,...,new_deaths_smoothed_per_million,reproduction_rate,icu_patients,icu_patients_per_million,hosp_patients,hosp_patients_per_million,weekly_icu_admissions,weekly_icu_admissions_per_million,weekly_hosp_admissions,weekly_hosp_admissions_per_million
0,DZA,Africa,Algeria,2/25/20,44616626,1,1,None,None,None,...,None,None,None,None,None,None,None,None,None,None
1,DZA,Africa,Algeria,2/26/20,44616626,1,0,None,None,None,...,None,None,None,None,None,None,None,None,None,None
2,DZA,Africa,Algeria,2/27/20,44616626,1,0,None,None,None,...,None,None,None,None,None,None,None,None,None,None
3,DZA,Africa,Algeria,2/28/20,44616626,1,0,None,None,None,...,None,None,None,None,None,None,None,None,None,None
4,DZA,Africa,Algeria,2/29/20,44616626,1,0,None,None,None,...,None,None,None,None,None,None,None,None,None,None


In [5]:
# print first 5 rows of covid_vaccinations table using pandas directly
query = """
    SELECT *
    FROM covid_vaccinations
    WHERE continent = 'Africa'
    LIMIT 5;
"""

df = pd.read_sql_query(query, engine)
df

,iso_code,continent,location,date,new_tests,total_tests,total_tests_per_thousand,new_tests_per_thousand,new_tests_smoothed,new_tests_smoothed_per_thousand,...,extreme_poverty,cardiovasc_death_rate,diabetes_prevalence,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,human_development_index,excess_mortality
0,DZA,Africa,Algeria,2/25/20,None,None,None,None,None,None,...,0.5,278.364,6.73,0.7,30.4,83.741,1.9,76.88,0.748,None
1,DZA,Africa,Algeria,2/26/20,None,None,None,None,None,None,...,0.5,278.364,6.73,0.7,30.4,83.741,1.9,76.88,0.748,None
2,DZA,Africa,Algeria,2/27/20,None,None,None,None,None,None,...,0.5,278.364,6.73,0.7,30.4,83.741,1.9,76.88,0.748,None
3,DZA,Africa,Algeria,2/28/20,None,None,None,None,None,None,...,0.5,278.364,6.73,0.7,30.4,83.741,1.9,76.88,0.748,None
4,DZA,Africa,Algeria,2/29/20,None,None,None,None,None,None,...,0.5,278.364,6.73,0.7,30.4,83.741,1.9,76.88,0.748,None


## Analysis of African Data

__10 countries with the highest death count__

In [6]:
query = """
    SELECT location, MAX(cast(total_deaths AS INT)) AS max_total_deaths
    FROM covid_deaths
    WHERE continent = 'Africa'
    GROUP BY location
    ORDER BY max_total_deaths DESC
    LIMIT 10;
"""

df = pd.read_sql_query(query, engine)
df

,location,max_total_deaths
0,South Africa,85002
1,Tunisia,24244
2,Egypt,16885
3,Morocco,13618
4,Algeria,5596
5,Ethiopia,4967
6,Kenya,4923
7,Zimbabwe,4543
8,Libya,4441
9,Zambia,3633


__10 countries with the highest contamination count__

In [7]:
query = """
    SELECT location, MAX(cast(total_cases AS INT)) AS max_total_cases
    FROM covid_deaths
    WHERE continent = 'Africa'
    GROUP BY location
    ORDER BY max_total_cases DESC
    LIMIT 10;
"""

df = pd.read_sql_query(query, engine)
df

,location,max_total_cases
0,South Africa,2860835
1,Morocco,905564
2,Tunisia,685799
3,Libya,325221
4,Ethiopia,323715
5,Egypt,293448
6,Kenya,243929
7,Zambia,207960
8,Algeria,200301
9,Nigeria,199538


__10 countries with the highest rate of death if contaminated__

In [8]:
query = """
    WITH max_deaths (location, max_total_deaths)
    AS
    (
        SELECT location, MAX(cast(total_deaths AS INT)) AS max_total_deaths
        FROM covid_deaths
        WHERE continent = 'Africa'
        GROUP BY location
    ),

    max_cases (location, max_total_cases)
    AS
    (
        SELECT location, MAX(cast(total_cases AS INT)) AS max_total_cases
        FROM covid_deaths
        WHERE continent = 'Africa'
        GROUP BY location
    )


    SELECT max_deaths.location, max_deaths.max_total_deaths, max_cases.max_total_cases,
        (CAST(max_deaths.max_total_deaths AS REAL) / CAST(max_cases.max_total_cases AS REAL)) * 100 AS death_percentage
    FROM max_cases
        JOIN max_deaths
        ON max_cases.location = max_deaths.location
    WHERE death_percentage <> "None"
    ORDER BY death_percentage DESC
    LIMIT 10;
"""

df = pd.read_sql_query(query, engine)
df

,location,max_total_deaths,max_total_cases,death_percentage
0,Sudan,2837,37951,7.475429
1,Egypt,16885,293448,5.754001
2,Somalia,1032,18568,5.557949
3,Liberia,245,5727,4.277981
4,Malawi,2242,61137,3.667174
5,Tanzania,50,1367,3.657644
6,Mali,543,14984,3.623865
7,Zimbabwe,4543,126399,3.594174
8,Comoros,147,4100,3.585366
9,Tunisia,24244,685799,3.535147


__10 countries with the highest rate of contamination__

In [9]:
query = """
    SELECT location, CAST(population AS INT) AS population, MAX(CAST(total_cases AS INT)) AS max_total_cases, 
        (MAX(CAST(total_cases AS REAL)) / MAX(CAST(population AS REAL))) * 100 AS contamination_percentage
    FROM covid_deaths
    WHERE continent = 'Africa'
    GROUP BY location
    ORDER BY contamination_percentage DESC
    LIMIT 10;
"""

df = pd.read_sql_query(query, engine)
df

,location,population,max_total_cases,contamination_percentage
0,Seychelles,98910,20593,20.819937
1,Botswana,2397240,165644,6.909780
2,Cape Verde,561901,36640,6.520722
3,Tunisia,11935764,685799,5.745749
4,Namibia,2587344,126341,4.883038
5,South Africa,60041996,2860835,4.764723
6,Libya,6958538,325221,4.673697
7,Eswatini,1172369,44976,3.836335
8,Morocco,37344787,905564,2.424874
9,Sao Tome and Principe,223364,2825,1.264752


__10 countries with the highest average new cases per day__

In [10]:
query = """
    SELECT location, AVG(CASt(new_cases AS INT)) AS average_new_cases
    FROM covid_deaths
    WHERE continent = 'Africa'
    GROUP BY location
    ORDER BY average_new_cases DESC
    LIMIT 10;
"""

df = pd.read_sql_query(query, engine)
df

,location,average_new_cases
0,South Africa,5126.944444
1,Morocco,1614.196078
2,Tunisia,1226.831843
3,Libya,603.378479
4,Ethiopia,588.572727
5,Egypt,507.695502
6,Kenya,443.507273
7,Zambia,381.577982
8,Nigeria,353.790780
9,Algeria,353.264550


__The single day with the highest new cases count for each country__

In [11]:
query = """
    SELECT location, date, MAX(CAST(new_cases AS INT)) AS max_new_cases
    FROM covid_deaths
    WHERE continent = 'Africa'
    GROUP BY location;
"""

df = pd.read_sql_query(query, engine)
df

,location,date,max_new_cases
0,Algeria,7/28/21,1927.0
1,Angola,5/26/21,405.0
2,Benin,9/1/21,2566.0
3,Botswana,8/2/21,8530.0
4,Burkina Faso,12/12/20,315.0
5,Burundi,9/10/21,1462.0
6,Cameroon,4/1/21,9668.0
7,Cape Verde,4/18/21,491.0
8,Central African Republic,8/10/21,4044.0
9,Chad,1/9/21,91.0


__10 days with the highest recorded number of new cases on the continent with the country associated with them__

In [12]:
query = """
    SELECT location, date, MAX(CAST(new_cases AS INT)) AS max_new_cases
    FROM covid_deaths
    WHERE continent = 'Africa'
    GROUP BY date
    ORDER BY max_new_cases DESC
    LIMIT 10;
"""

df = pd.read_sql_query(query, engine)
df

,location,date,max_new_cases
0,South Africa,7/3/21,26485
1,South Africa,7/2/21,24270
2,South Africa,7/8/21,22910
3,South Africa,7/9/21,22441
4,South Africa,1/8/21,21980
5,South Africa,1/6/21,21832
6,South Africa,7/10/21,21610
7,South Africa,1/9/21,21606
8,South Africa,7/1/21,21584
9,South Africa,7/7/21,21427


__10 countries leading the vaccination effort on the continent with their death and contamination counts__

In [13]:
query = """
    SELECT covid_vaccinations.location, MAX(CAST(covid_vaccinations.total_tests AS INT)) AS max_total_tests, 
        MAX(cast(covid_deaths.total_deaths AS INT)) AS max_total_deaths, 
        MAX(cast(covid_deaths.total_cases AS INT)) AS max_total_cases
    FROM covid_vaccinations
        JOIN covid_deaths
        ON covid_vaccinations.location = covid_deaths.location
    WHERE covid_vaccinations.continent = 'Africa' AND covid_deaths.continent = 'Africa'
    GROUP BY covid_vaccinations.location
    ORDER BY max_total_tests DESC
    LIMIT 10;
"""

df = pd.read_sql_query(query, engine)
df

,location,max_total_tests,max_total_deaths,max_total_cases
0,South Africa,17073236,85002,2860835
1,Morocco,8459324,13618,905564
2,Ethiopia,3330057,4967,323715
3,Nigeria,2942578,2619,199538
4,Tunisia,2812338,24244,685799
5,Rwanda,2593862,1175,93079
6,Zambia,2355773,3633,207960
7,Kenya,1675310,4923,243929
8,Uganda,1626546,3099,121587
9,Ghana,1422916,1098,123874


__10 countries with the highest hospital capacities on the continent and their death rates from covid-19__

In [14]:
query = """
    WITH max_deaths (location, max_total_deaths)
    AS
    (
        SELECT location, MAX(cast(total_deaths AS INT)) AS max_total_deaths
        FROM covid_deaths
        WHERE continent = 'Africa'
        GROUP BY location
    )

    SELECT covid_vaccinations.location, covid_deaths.population, max_deaths.max_total_deaths,
        MAX(covid_vaccinations.hospital_beds_per_thousand) AS max_hospital_beds_per_thousand,
        (CAST(max_deaths.max_total_deaths AS REAL) / CAST(covid_deaths.population AS REAL)) * 1000 AS death_per_thousand
    FROM max_deaths
        JOIN covid_vaccinations
        ON max_deaths.location = covid_vaccinations.location
        JOIN covid_deaths
        ON max_deaths.location = covid_deaths.location
    GROUP BY max_deaths.location
    ORDER BY max_hospital_beds_per_thousand DESC
    LIMIT 10;
"""

df = pd.read_sql_query(query, engine)
df

,location,population,max_total_deaths,max_hospital_beds_per_thousand,death_per_thousand
0,Gabon,2278829,173,6.30,0.075916
1,Libya,6958538,4441,3.70,0.638209
2,Seychelles,98910,106,3.60,1.071681
3,Mauritius,1273428,43,3.40,0.033767
4,Sao Tome and Principe,223364,41,2.90,0.183557
5,South Africa,60041996,85002,2.32,1.415709
6,Tunisia,11935764,24244,2.30,2.031206
7,Comoros,888456,147,2.20,0.165456
8,Eswatini,1172369,1179,2.10,1.005656
9,Equatorial Guinea,1449891,131,2.10,0.090352
